In [11]:
from bs4 import BeautifulSoup
from docx import Document

def convert_html_to_docx(html_file, output_docx):
    # Load HTML content from file
    with open(html_file, 'r', encoding='utf-8') as f:
        html_content = f.read()

    # Parse HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Create a new Word document
    doc = Document()

    # Find all text within <div> tags in the HTML
    text_divs = soup.find_all('div')

    # Loop through each div and add its text to the Word document
    for div in text_divs:
        # Add text content of the div
        doc.add_paragraph(div.get_text())

    # Save the Word document
    doc.save(output_docx)

# Example usage:
convert_html_to_docx('index.html', 'output.docx')


In [13]:
from bs4 import BeautifulSoup
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

def convert_html_to_docx(html_file, output_docx):
    # Load HTML content from file
    with open(html_file, 'r', encoding='utf-8') as f:
        html_content = f.read()

    # Parse HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Create a new Word document
    doc = Document()

    # Find all <div> tags in the HTML
    divs = soup.find_all('div')

    # Loop through each <div> tag
    for div in divs:
        # Extract text content from the <div> tag
        text = div.get_text(strip=True)

        # Skip empty text
        if not text:
            continue

        # Add text to the Word document
        paragraph = doc.add_paragraph(text)

        # Apply font styles from the <div> tag's style attribute, if present
        if 'style' in div.attrs:
            styles = div['style'].split(';')
            for style in styles:
                key, value = style.split(':')
                if key.strip() == 'font-size':
                    paragraph.style.font.size = Pt(float(value.strip().replace('px', '')))  # Convert from px to points
                elif key.strip() == 'font-family':
                    paragraph.style.font.name = value.strip()
                elif key.strip() == 'text-align':
                    alignment = value.strip()
                    if alignment == 'center':
                        paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
                    elif alignment == 'right':
                        paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.RIGHT
                    else:
                        paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT

        # Find all <img> tags within the <div> tag
        imgs = div.find_all('img')
        for img in imgs:
            # Extract image URL from the src attribute
            img_url = img['src']
            # Add image to the Word document
            doc.add_picture(img_url)

    # Save the Word document
    doc.save(output_docx)

# Example usage:
convert_html_to_docx('index.html', 'output.docx')


In [7]:
from bs4 import BeautifulSoup
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

def convert_html_to_docx(html_file, output_docx):
    # Load HTML content from file
    with open(html_file, 'r', encoding='utf-8') as f:
        html_content = f.read()

    # Parse HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Create a new Word document
    doc = Document()

    # Find all <div> tags in the HTML
    divs = soup.find_all('div')

    # Loop through each <div> tag
    for div in divs:
        # Extract text content from the <div> tag
        text = div.get_text(strip=True)

        # Skip empty text
        if not text:
            continue

        # Add text to the Word document
        paragraph = doc.add_paragraph(text)

        # Apply font styles from the <div> tag's style attribute, if present
        if 'style' in div.attrs:
            styles = div['style'].split(';')
            for style in styles:
                key, value = style.split(':')
                if key.strip() == 'font-size':
                    paragraph.style.font.size = Pt(float(value.strip().replace('px', '')))  # Convert from px to points
                elif key.strip() == 'font-family':
                    paragraph.style.font.name = value.strip()
                elif key.strip() == 'text-align':
                    alignment = value.strip()
                    if alignment == 'center':
                        paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
                    elif alignment == 'right':
                        paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.RIGHT
                    else:
                        paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT

    # Save the Word document
    doc.save(output_docx)

# Example usage:
convert_html_to_docx('index.html', 'output.docx')


In [9]:
from bs4 import BeautifulSoup
from docx import Document
from docx.shared import Inches

def convert_html_to_docx(html_file, output_docx):
    # Load HTML content from file
    with open(html_file, 'r', encoding='utf-8') as f:
        html_content = f.read()

    # Parse HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Create a new Word document
    doc = Document()

    # Find all <div> tags in the HTML
    divs = soup.find_all('div')

    # Loop through each <div> tag
    for div in divs:
        # Extract text content from the <div> tag
        text = div.get_text(strip=True)

        # Skip empty text
        if not text:
            continue

        # Add text to the Word document
        paragraph = doc.add_paragraph(text)

        # Apply font styles from the <div> tag's style attribute, if present
        if 'style' in div.attrs:
            styles = div['style'].split(';')
            for style in styles:
                key, value = style.split(':')
                if key.strip() == 'font-size':
                    paragraph.style.font.size = Pt(float(value.strip().replace('px', '')))  # Convert from px to points
                elif key.strip() == 'font-family':
                    paragraph.style.font.name = value.strip()
                elif key.strip() == 'text-align':
                    alignment = value.strip()
                    if alignment == 'center':
                        paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
                    elif alignment == 'right':
                        paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.RIGHT
                    else:
                        paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT

    # Find all <img> tags in the HTML
    imgs = soup.find_all('img')

    # Loop through each <img> tag
    for img in imgs:
        # Extract image URL from the src attribute
        img_url = img['src']
        
        # Add image to the Word document
        doc.add_picture(img_url, width=Inches(3))  # Adjust width as needed

    # Save the Word document
    doc.save(output_docx)

# Example usage:
convert_html_to_docx('index.html', 'output.docx')


In [10]:
from bs4 import BeautifulSoup
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
import os

def parse_css(css_file):
    # Load CSS content from file
    with open(css_file, 'r', encoding='utf-8') as f:
        css_content = f.read()

    # Parse CSS content and extract relevant styles
    styles = {}
    for line in css_content.splitlines():
        if '{' in line and '}' in line:
            selector, props = line.split('{')
            selector = selector.strip()
            props = props.strip().rstrip('}')
            styles[selector] = {}
            for prop in props.split(';'):
                if ':' in prop:
                    key, value = prop.split(':')
                    styles[selector][key.strip()] = value.strip()

    return styles

def apply_styles(element, docx_element, styles):
    # Apply styles from CSS file to the element in the Word document
    for selector, props in styles.items():
        if selector in element['class']:
            for key, value in props.items():
                if key == 'font-size':
                    docx_element.style.font.size = Pt(float(value.replace('px', '')))
                elif key == 'font-family':
                    docx_element.style.font.name = value
                elif key == 'text-align':
                    if value == 'center':
                        docx_element.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
                    elif value == 'right':
                        docx_element.alignment = WD_PARAGRAPH_ALIGNMENT.RIGHT
                    else:
                        docx_element.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT

def convert_html_to_docx(html_file, css_file, output_docx):
    # Load HTML content from file
    with open(html_file, 'r', encoding='utf-8') as f:
        html_content = f.read()

    # Parse HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Load CSS styles
    styles = parse_css(css_file)

    # Create a new Word document
    doc = Document()

    # Find all <div> tags in the HTML
    divs = soup.find_all('div')

    # Loop through each <div> tag
    for div in divs:
        # Extract text content from the <div> tag
        text = div.get_text(strip=True)

        # Skip empty text
        if not text:
            continue

        # Add text to the Word document
        paragraph = doc.add_paragraph(text)

        # Apply font styles from the CSS file based on class attribute of the <div> tag
        if 'class' in div.attrs:
            apply_styles(div, paragraph, styles)

    # Find all <img> tags in the HTML
    imgs = soup.find_all('img')

    # Loop through each <img> tag
    for img in imgs:
        # Extract image URL from the src attribute
        img_url = img['src']
        
        # Add image to the Word document
        doc.add_picture(img_url)

    # Save the Word document
    doc.save(output_docx)

# Example usage:
html_file = 'index.html'
css_file = os.path.join('css', 'style.css')
output_docx = 'output.docx'
convert_html_to_docx(html_file, css_file, output_docx)
